# Setup

In [ ]:
!pip install -U pip watermark -q --root-user-action=ignore

## Imports

In [ ]:
import base64
import json
import os
import time
from io import BytesIO

import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.pipeline import PipelineModel
from sagemaker.s3 import S3Downloader
from sagemaker.serverless.serverless_inference_config import ServerlessInferenceConfig
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.transformer import Transformer
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

## Versions

In [ ]:
%load_ext watermark
%watermark --iversions

## Parameters

In [ ]:
bucket = "datarocket-stg-sagemaker"
prefix = "california-housing"


sm_region = boto3.Session().region_name
sm_role = sagemaker.get_execution_role()
sm_session = sagemaker.Session(default_bucket=bucket)
sm_client = boto3.client("sagemaker", sm_region)
runtime = boto3.client("sagemaker-runtime")


data = fetch_california_housing()
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)
testX = pd.DataFrame(X_test, columns=data.feature_names)


model_file_name = "model.joblib"
model_name = "california-housing"
serverless_inference_config = ServerlessInferenceConfig(memory_size_in_mb=2048, max_concurrency=10)
endpoint_name_serverless = "california-housing-serverless"
endpoint_name_realtime = "california-housing-realtime"
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=f"s3://{bucket}/{prefix}/data_capture",
)

## Data

In [ ]:
testX.shape

In [ ]:
testX.head()

In [ ]:
testX[data.feature_names].to_csv("./data/california_housing.csv", header=False, index=False)

sm_session.upload_data(
    path="./data/california_housing.csv", bucket=bucket, key_prefix=f"{prefix}/batch_data"
)

batch_data = f"s3://{bucket}/{prefix}/batch_data/"
batch_data

## Model Data

In [ ]:
!tar czvf model.tar.gz $model_file_name

In [ ]:
fObj = open("model.tar.gz", "rb")
key = os.path.join(prefix, "model.tar.gz")
boto3.Session().resource("s3").Bucket(bucket).Object(key).upload_fileobj(fObj)

In [ ]:
model_data = "s3://{}/{}".format(bucket, key)
model_data

## Model Code

In [ ]:
!pygmentize ./code/inference.py

In [ ]:
!pygmentize ./code/requirements.txt

# SageMaker

## SKLearnModel

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#scikit-learn-model

model = SKLearnModel(
    name=model_name,
    role=sm_role,
    model_data=model_data,
    framework_version="1.0-1",
    py_version="py3",
    source_dir="code",
    entry_point="inference.py",
    sagemaker_session=sm_session,
)

## SageMaker Model

In [ ]:
model.create()

## Model Registry

In [ ]:
register = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    transform_instances=["ml.m5.large", "ml.m5.xlarge"],
    model_package_group_name=model_name,
    approval_status="Approved",
    description="My sample California housing model package group"
    # model_metrics=model_metrics,
)

## Batch Transform

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type="ml.m5.large",
    max_payload=10,
    accept="text/csv",
    assemble_with="Line",
    output_path=f"s3://{bucket}/{prefix}/result/",
)

In [ ]:
transformer.transform(
    batch_data,
    join_source="Input",
    split_type="Line",
    content_type="text/csv",
)

In [ ]:
# Download the output data from S3 to local file system
batch_output = transformer.output_path
!mkdir -p data/output
!aws s3 cp --recursive $batch_output data/output/
# Head to see what the batch output looks like
!head data/output/*

In [ ]:
feature_names = data.feature_names.copy()
feature_names.append("predict")

pd.read_csv(
    "data/output/california_housing.csv.out",
    header=None,
    names=feature_names,
)

## Deploy Endpoint - Serverless

In [ ]:
predictor_serverless = model.deploy(
    endpoint_name=endpoint_name_serverless,
    serverless_inference_config=serverless_inference_config,
)

In [ ]:
predictor_realtime = model.deploy(
    endpoint_name=endpoint_name_realtime,
    instance_type="ml.t2.medium",
    initial_instance_count=1,
    data_capture_config=data_capture_config,
)

## Predictor

In [ ]:
predictions_realtime = predictor_realtime.predict(testX[data.feature_names])
predictions_serverless = predictor_realtime.predict(testX[data.feature_names])

df_predictions_realtime = pd.DataFrame(predictions_realtime, columns=["predict"])
df_predictions_serverless = pd.DataFrame(predictions_serverless, columns=["predict"])

In [ ]:
df_predictions_realtime.shape, df_predictions_serverless.shape

## Predictions Realtime

In [ ]:
df_predictions_realtime

## Predictions Serverless

In [ ]:
df_predictions_serverless

## Invoke Endpoint (application/x-npy)

In [ ]:
# line 59 - https://github.com/aws/sagemaker-scikit-learn-container/blob/master/src/sagemaker_sklearn_container/serving.py
# lines 191, 48 - https://github.com/aws/sagemaker-training-toolkit/blob/master/src/sagemaker_training/encoders.py

# Serialise numpy ndarray as bytes
buffer = BytesIO()
np.save(buffer, testX[data.feature_names].values)

x_npy_response_realtime = runtime.invoke_endpoint(
    EndpointName=predictor_realtime.endpoint_name,
    Body=buffer.getvalue(),
    ContentType="application/x-npy",
)

x_npy_response_serverless = runtime.invoke_endpoint(
    EndpointName=predictor_serverless.endpoint_name,
    Body=buffer.getvalue(),
    ContentType="application/x-npy",
)

x_npy_predictions_serverless = json.loads(x_npy_response_serverless["Body"].read())
x_npy_predictions_realtime = json.loads(x_npy_response_realtime["Body"].read())

df_x_npy_predictions_realtime = pd.DataFrame(x_npy_predictions_realtime, columns=["Predict"])
df_x_npy_predictions_serverless = pd.DataFrame(x_npy_predictions_serverless, columns=["Predict"])

In [ ]:
df_x_npy_predictions_serverless.shape, df_x_npy_predictions_realtime.shape

## Invoke Endpoint (text/csv)

In [ ]:
csv_response_realtime = runtime.invoke_endpoint(
    EndpointName=predictor_realtime.endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

csv_response_serverless = runtime.invoke_endpoint(
    EndpointName=predictor_serverless.endpoint_name,
    Body=testX[data.feature_names].to_csv(header=False, index=False).encode("utf-8"),
    ContentType="text/csv",
)

csv_predictions_serverless = json.loads(csv_response_serverless["Body"].read())
csv_predictions_realtime = json.loads(csv_response_realtime["Body"].read())

df_csv_predictions_realtime = pd.DataFrame(csv_predictions_realtime, columns=["Predict"])
df_csv_predictions_serverless = pd.DataFrame(csv_predictions_serverless, columns=["Predict"])

In [ ]:
df_csv_predictions_realtime.shape, df_csv_predictions_serverless.shape

## Capture realtime

In [ ]:
# https://sagemaker.readthedocs.io/en/stable/amazon_sagemaker_model_monitoring.html
# https://github.com/aws/amazon-sagemaker-examples/blob/main/aws_sagemaker_studio/getting_started/xgboost_customer_churn_studio.ipynb
# https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker_model_monitor/model_monitor_batch_transform

for _ in range(20):  # wait up to a minute to see captures in S3
    capture_files = S3Downloader.list(f"s3://{bucket}/{prefix}/data_capture")
    if capture_files:
        break
    time.sleep(5)

print("Found Data Capture Files:")
print(capture_files)

In [ ]:
capture_file = S3Downloader.read_file(capture_files[-1])

In [ ]:
print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2)[:1000])

## Delete Resources

In [ ]:
def empty_and_delete_model_package(sagemaker_client, mpg_name):
    mpg = sagemaker_client.list_model_packages(
        ModelPackageGroupName=mpg_name,
    )

    # Delete model packages if Group not empty
    model_packages = mpg.get("ModelPackageSummaryList")
    if model_packages:
        for mp in model_packages:
            sagemaker_client.delete_model_package(ModelPackageName=mp["ModelPackageArn"])
            time.sleep(1)

    # Delete model package group
    sagemaker_client.delete_model_package_group(ModelPackageGroupName=mpg_name)


model.delete_model()
predictor_realtime.delete_endpoint(delete_endpoint_config=True)
predictor_serverless.delete_endpoint(delete_endpoint_config=True)
empty_and_delete_model_package(sm_client, model_name)